# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")


# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Myanaung,18.2833,95.3167,31.23,36,63,1.81,MM,1680059482
1,1,Cherskiy,68.7500,161.3000,-7.69,77,100,8.94,RU,1680059482
2,2,Cabo San Lucas,22.8909,-109.9124,21.95,61,40,1.54,MX,1680059483
3,3,George Town,5.4112,100.3354,30.96,74,20,2.57,MY,1680059483
4,4,Ilulissat,69.2167,-51.1000,-27.42,89,5,2.24,GL,1680059484


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [39]:


# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng","Lat", geo=True,tiles="OSM",frame_width = 800, frame_height = 600, size = "Humidity")

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_city_data = city_data_df.loc[(city_data_df["Max Temp"]>=21) & (city_data_df["Max Temp"]<=27) & (city_data_df["Wind Speed"]<4.5) & (city_data_df["Cloudiness"]==0)]

# Drop any rows with null values
narrow_city_data = narrow_city_data.dropna()

# Display sample data
narrow_city_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
37,37,Tiznit Province,29.5833,-9.5000,22.43,14,0,2.07,MA,1680059497
93,93,Arlit,18.7369,7.3853,22.53,10,0,4.26,NE,1680059459
128,128,Caravelas,-17.7125,-39.2481,23.95,82,0,1.93,BR,1680059535
149,149,Ixtapa,20.7000,-105.2000,22.57,78,0,2.06,MX,1680059252
368,368,Concordia,-31.3930,-58.0209,23.21,74,0,1.79,AR,1680059386
387,387,Nizwá,22.9333,57.5333,21.58,46,0,1.77,OM,1680059651
491,491,Māngrol,21.1167,70.1167,24.40,67,0,3.13,IN,1680059693
516,516,Verāval,20.9000,70.3667,24.53,66,0,3.10,IN,1680059703
543,543,Porbandar,21.6422,69.6093,23.96,69,0,2.59,IN,1680059714


### Step 3: Create a new DataFrame called `hotel_df`.

In [25]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_city_data[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
37,Tiznit Province,MA,29.5833,-9.5000,14,
93,Arlit,NE,18.7369,7.3853,10,
128,Caravelas,BR,-17.7125,-39.2481,82,
149,Ixtapa,MX,20.7000,-105.2000,78,
368,Concordia,AR,-31.3930,-58.0209,74,
387,Nizwá,OM,22.9333,57.5333,46,
491,Māngrol,IN,21.1167,70.1167,67,
516,Verāval,IN,20.9000,70.3667,66,
543,Porbandar,IN,21.6422,69.6093,69,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [32]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,"Lat"]
    longitude = hotel_df.loc[index,"Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Tiznit Province - nearest hotel: No hotel found
Arlit - nearest hotel: No hotel found
Caravelas - nearest hotel: Pousada dos Navegantes
Ixtapa - nearest hotel: Ma. Cristina
Concordia - nearest hotel: D'Charruas
Nizwá - nearest hotel: Nizwa Heritage Inn
Māngrol - nearest hotel: No hotel found
Verāval - nearest hotel: Shyam
Porbandar - nearest hotel: Toran Tourist Bungalow


,City,Country,Lat,Lng,Humidity,Hotel Name
37,Tiznit Province,MA,29.5833,-9.5000,14,No hotel found
93,Arlit,NE,18.7369,7.3853,10,No hotel found
128,Caravelas,BR,-17.7125,-39.2481,82,Pousada dos Navegantes
149,Ixtapa,MX,20.7000,-105.2000,78,Ma. Cristina
368,Concordia,AR,-31.3930,-58.0209,74,D'Charruas
387,Nizwá,OM,22.9333,57.5333,46,Nizwa Heritage Inn
491,Māngrol,IN,21.1167,70.1167,67,No hotel found
516,Verāval,IN,20.9000,70.3667,66,Shyam
543,Porbandar,IN,21.6422,69.6093,69,Toran Tourist Bungalow


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [35]:


# Configure the map plot
map_plot1 = hotel_df.hvplot.points("Lng","Lat", geo=True,tiles="OSM",frame_width = 800, frame_height = 600, hover_cols = ["Hotel Name", "Country"])

# Display the map
map_plot1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)